In [3]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import plotly.express as px 
import ipywidgets as widgets
import numpy as np
from scipy import special
import time
import math as ms

#google cloud big query libraries
from google.cloud import bigquery
from google.oauth2 import service_account

if os.getlogin()=='cday':
    key_path = r"C:\Users\cday\tdm-scenarios-c90ba30c3c5d.json"
elif os.getlogin()=='bhereth':
    key_path = r"C:\Users\bhereth\tdm-scenarios-a85044dbbfd3.json"
elif os.getlogin()=='sswim':
    key_path = r"C:\Users\sswim\tdm-scenarios-a5ce45fae4b0.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
 
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [4]:
# include in all scenario groups
lstIncludeInAll = ['BY','TIP']

# dataframe to create subcategories
dfModeGroups = pd.DataFrame([
    ['Walk'     ,'1: Non-Motorized'         ],
    ['Bike'     ,'1: Non-Motorized'         ],
    ['Walk'     ,'1a: Walk'                 ],
    ['Bike'     ,'1b: Bike'                 ],
    ['Auto'     ,'2: Auto'                  ],
    ['SchoolBus','3: SchoolBus'             ],
    ['LCL'      ,'4: Transit'               ],
    ['COR'      ,'4: Transit'               ],
    ['EXP'      ,'4: Transit'               ],
    ['BRT'      ,'4: Transit'               ],
    ['LRT'      ,'4: Transit'               ],
    ['CRT'      ,'4: Transit'               ],
    ['LCL'      ,'4a: Local Bus'            ],
    ['COR'      ,'4b: Core Bus'             ],
    ['EXP'      ,'4c: Express Bus'          ],
    ['BRT'      ,'4d: Bus-Rapid Transit'    ],
    ['LRT'      ,'4e: Light-Rail Transit'   ],
    ['CRT'      ,'4f: Commuter-Rail Transit'],
    ['Walk'     ,'0: Total'                 ],
    ['Bike'     ,'0: Total'                 ],
    ['Walk'     ,'0: Total'                 ],
    ['Bike'     ,'0: Total'                 ],
    ['Auto'     ,'0: Total'                 ],
    ['SchoolBus','0: Total'                 ],
    ['LCL'      ,'0: Total'                 ],
    ['COR'      ,'0: Total'                 ],
    ['EXP'      ,'0: Total'                 ],
    ['BRT'      ,'0: Total'                 ],
    ['LRT'      ,'0: Total'                 ],
    ['CRT'      ,'0: Total'                 ],
    ['LCL'      ,'0: Total'                 ],
    ['COR'      ,'0: Total'                 ],
    ['EXP'      ,'0: Total'                 ],
    ['BRT'      ,'0: Total'                 ],
    ['LRT'      ,'0: Total'                 ],
    ['CRT'      ,'0: Total'                 ]]
,columns=('MODE','modeGroup'))
#dfModeGroups

In [5]:
# merge scenario data onto transit share data

# read transit summary from biq query
dfTransitSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.transit_share").to_dataframe()
#display(dfTransitSummary)

# read scenarios data biq query
dfScenarios = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios").to_dataframe()
#display(dfScenarios)

# merge two dataframes
dfTransitSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfTransitSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge to mode groupings dataframe
dfTransitSummaryPlotData = pd.DataFrame.merge(dfTransitSummaryWithScenarioData,dfModeGroups,on='MODE')
#dfTransitSummaryPlotData

In [6]:
# check to see if all scenarios has same record count
dfTransitSummaryPlotData_Check = dfTransitSummaryPlotData
dfTransitSummaryPlotData_Check.groupby(['tdmVersionWithDate','scenarioGroup','scenarioYear']).agg(COUNT=('TRIPS','count'))

COUNT
tdmVersionWithDate          scenarioGroup scenarioYear       
WF TDM v8.3.2 - 2022-02-04a BY            2019            360
                            RTP           2030            360
                                          2040            360
                                          2050            360
                            TIP           2024            360
WF TDM v8.3.2 - 2022-09-21  BY            2019            360
                                          2023            360
                            Needs         2032            360
                                          2042            360
                                          2050            360
                            NoBuild       2032            360
                                          2042            360
                                          2050            360
                            RTP           2032            360
                                          2042            360
                                          2050            360
                            TIP           2028            360
WF TDM v9.0 - 2022-12-19    BY            2019            360
                                          2023            360
                            EF_RTP        2032            360
                                          2042            360
                                          2050            360
                            Needs         2032            360
                                          2042            360
                                          2050            360
                            NoBuild       2032            360
                                          2042            360
                                          2050            360
                            RTP           2032            360
                                          2042            360
                                          2050            360
                            TIP           2028            360

In [7]:
#PLOTTING FUNCTION

import math

def update_plot(tdmVersionsWithDate, scenarioGroups, modeGroups, trippurps, periods):

    data = []

    for v in tdmVersionsWithDate:
        for g in scenarioGroups:
            for m in modeGroups:

                # only do if data in dataframe since BY data is concatonated later
                if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin([g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))].shape[0]>1:

                    # data for plotting from filtered dataframe}
                    plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin(lstIncludeInAll + [g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))]

                    #display(plotdata)

                    plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                    # fill any NaN values with zeros
                    plotdata = plotdata.fillna(0)

                    #display(plotdata)

                    xplot = plotdata['scenarioYear']
                    yplot = plotdata['TRIPS'       ]

                    trace1 = go.Scatter(
                        x=xplot,
                        y=yplot,
                        mode='markers+lines',
                        name= v + ' ' + g + ' ' + m,
                        marker=dict(size=12,
                                line=dict(width=2,
                                            color='DarkSlateGrey'))
        #                line=dict(
        #                    shape='spline'
        #                )
                    )
                    data.append(trace1)


    layout = go.Layout(
        title='Trips by Mode (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Trips',
            rangemode = 'tozero'#,
            #range=(0,np.null)
        ),
        xaxis=dict(
            title='Year',
            range=(2018,2051)
        ),
        width=1200,
        height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [62]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTdmVersionWithDate = dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist()
lstScenarioGroups     = dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist()
lstModes              = dfTransitSummaryPlotData['modeGroup'         ].unique().tolist()
lstTripPurp           = dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist()
lstPeriod             = dfTransitSummaryPlotData['PERIOD'            ].unique().tolist()

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionsWithDate = widgets.SelectMultiple(options=lstTdmVersionWithDate, value=(lstTdmVersionWithDate), description='TDM Version'   )
selectScenarioGroups      = widgets.SelectMultiple(options=lstScenarioGroups    , value=('RTP',               ), description='Scenario Group')
selectModes               = widgets.SelectMultiple(options=lstModes             , value=('4: Transit',        ), description='Mode Group'    )
selectTripPurps           = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp          ), description='Purpose'       )
selectPeriods             = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod            ), description='Periods'       )

#custom extents
#xmin = widgets.Text(value='0'  , description='X Min')
#xmax = widgets.Text(value='160' , description='X Max')
#ymin = widgets.Text(value='0'  , description='Y Min')
#ymax = widgets.Text(value='.10', description='Y Max')

widgets.interactive(update_plot, tdmVersionsWithDate=selectTdmVersionsWithDate, scenarioGroups=selectScenarioGroups, modeGroups=selectModes, trippurps=selectTripPurps , periods=selectPeriods)

#ADD RAW OBSERVED CURVES

interactive(children=(SelectMultiple(description='TDM Version', index=(0, 1, 2), options=('WF TDM v8.3.2 - 202…

In [65]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea(tdmVersionWithDate, scenarioGroup, modeGroups, trippurps, periods):

    data = []

    modeGroups = sorted(modeGroups)

    for m in modeGroups: 
        # only do if data in dataframe since BY data is concatonated later
        if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate) & (dfTransitSummaryPlotData['scenarioGroup']==scenarioGroup) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))].shape[0]>1:

            # data for plotting from filtered dataframe}
            plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate) & (dfTransitSummaryPlotData['scenarioGroup'].isin(lstIncludeInAll + [scenarioGroup])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))]

            #display(plotdata)

            plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)

            #display(plotdata)

            xplot = plotdata['scenarioYear']
            yplot = plotdata['TRIPS'       ]

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name= m,
                stackgroup='one'#,
                #groupnorm='percent' # sets the normalization for the sum of the stackgroup
            )
            data.append(trace1)


    layout = go.Layout(
        title=tdmVersionWithDate + ' Trips Mode Split (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Trips'#,
            #rangemode = 'tozero',
            #ticksuffix='%',
            #range=(0,100)
        ),
        xaxis=dict(
            title='Year'#,
            #range=(2018,2051)
        ),
        width=1000,
        height=450
    )
    
    fig2 = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig2)


In [66]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

from ipywidgets import Layout

lstTdmVersionsWithDate = sorted(dfTransitSummaryPlotData['tdmVersionWithDate'].unique().tolist())
lstScenarioGroups      = sorted(dfTransitSummaryPlotData['scenarioGroup'     ].unique().tolist())
lstModes               = sorted(dfTransitSummaryPlotData['modeGroup'         ].unique().tolist())
lstTripPurp            = sorted(dfTransitSummaryPlotData['TRIPPURP'          ].unique().tolist())
lstPeriod              = sorted(dfTransitSummaryPlotData['PERIOD'            ].unique().tolist())

# remove include in all list
lstScenarioGroups = [i for i in lstScenarioGroups if i not in lstIncludeInAll] 

selectTdmVersionWithDate = widgets.Select        (options=lstTdmVersionWithDate, value=(lstTdmVersionsWithDate[2]          ), description='TDM Version'   , layout=Layout(display="flex", flex_flow='row'))
selectScenarioGroup      = widgets.Select        (options=lstScenarioGroups    , value=('RTP'                              ), description='Scenario Group', layout=Layout(display="flex", flex_flow='row'))
selectModes              = widgets.SelectMultiple(options=lstModes             , value=(lstModes[7:13]                     ), description='Mode Group'    , layout=Layout(display="flex", flex_flow='row'))
selectTripPurps          = widgets.SelectMultiple(options=lstTripPurp          , value=(lstTripPurp                        ), description='Purpose'       , layout=Layout(display="flex", flex_flow='row'))
selectPeriods            = widgets.SelectMultiple(options=lstPeriod            , value=(lstPeriod                          ), description='Periods'       , layout=Layout(display="flex", flex_flow='row'))

widgets.interactive(update_plot_stackedarea, tdmVersionWithDate=selectTdmVersionWithDate, scenarioGroup=selectScenarioGroup, modeGroups=selectModes, trippurps=selectTripPurps , periods=selectPeriods)

#ADD RAW OBSERVED CURVES

interactive(children=(Select(description='TDM Version', index=2, layout=Layout(display='flex', flex_flow='row'…